<a href="https://colab.research.google.com/github/tommyEzreal/study_2/blob/main/BERT_MLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! nvidia-smi

Fri Jan  6 10:42:10 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   60C    P0    28W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 85.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 103.0 MB/s eta 0:00:00


In [2]:
from transformers import BertTokenizer, BertForMaskedLM
import torch
from tqdm.auto import tqdm

In [4]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-multilingual-uncased')

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


### Masked Language modeling 

In [3]:
text = ("서광욱은 1997년 2월 13일에 태어났다. 그의 직업은 아직 학생",
        "이며 2023년 현재 AI와 DeepLearning을 공부하고 있다. 그는",
        "한국외국어대학교에 재학중이며 현재는 휴학중이다. 2023년 봄학기에",
        "복학이 예정되어있다.")

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
inputs = tokenizer(text, return_tensors='pt',padding=True)
inputs.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [8]:
inputs.input_ids

tensor([[  101,  1172, 33645, 12211, 13045, 27235, 34653, 51472, 18014, 46123,
         10609, 84416,   119, 21935,  1175, 32035, 12756, 29999, 11375,  1174,
         25539, 40419,  1181, 93871, 24247,   102,     0],
        [  101, 12398, 19724, 21526, 10959, 11084, 26841, 11279, 12747, 15526,
         21841, 20773, 10422, 10946, 47468, 13045, 15245, 12727, 11724,   119,
         18287,   102,     0,     0,     0,     0,     0],
        [  101, 47529, 74831, 20687, 13413, 30462, 10609,  1175, 26179, 21596,
         43312, 35165, 82227,  1181, 36581, 21596, 43312, 12362,   119, 21526,
         10959, 11084,  1170, 50823, 21596, 35913,   102],
        [  101,  1170, 70724, 21596, 11112,  1174, 61701, 17861, 16584, 74190,
           119,   102,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0]])

In [9]:
# create labels

In [10]:
inputs['labels'] = inputs.input_ids.detach().clone()

In [11]:
inputs

{'input_ids': tensor([[  101,  1172, 33645, 12211, 13045, 27235, 34653, 51472, 18014, 46123,
         10609, 84416,   119, 21935,  1175, 32035, 12756, 29999, 11375,  1174,
         25539, 40419,  1181, 93871, 24247,   102,     0],
        [  101, 12398, 19724, 21526, 10959, 11084, 26841, 11279, 12747, 15526,
         21841, 20773, 10422, 10946, 47468, 13045, 15245, 12727, 11724,   119,
         18287,   102,     0,     0,     0,     0,     0],
        [  101, 47529, 74831, 20687, 13413, 30462, 10609,  1175, 26179, 21596,
         43312, 35165, 82227,  1181, 36581, 21596, 43312, 12362,   119, 21526,
         10959, 11084,  1170, 50823, 21596, 35913,   102],
        [  101,  1170, 70724, 21596, 11112,  1174, 61701, 17861, 16584, 74190,
           119,   102,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0]

In [12]:
# use pytorch random - equal dim to input_ids tensor

rand = torch.rand(inputs.input_ids.shape)
rand.shape, inputs.input_ids.shape

(torch.Size([4, 27]), torch.Size([4, 27]))

In [13]:
rand

tensor([[0.6947, 0.9465, 0.8956, 0.5289, 0.6362, 0.9934, 0.9310, 0.7512, 0.7073,
         0.7579, 0.7280, 0.5733, 0.8950, 0.4438, 0.8338, 0.2025, 0.3073, 0.1147,
         0.1302, 0.8093, 0.0118, 0.7156, 0.5294, 0.4427, 0.5394, 0.4767, 0.8873],
        [0.7067, 0.4384, 0.6616, 0.9865, 0.5270, 0.2780, 0.4022, 0.7739, 0.0503,
         0.9817, 0.5560, 0.0174, 0.5945, 0.1289, 0.6185, 0.1436, 0.3806, 0.7658,
         0.9116, 0.7593, 0.3567, 0.8296, 0.0877, 0.3934, 0.6510, 0.0929, 0.7330],
        [0.6376, 0.3603, 0.1989, 0.1264, 0.0020, 0.4911, 0.5962, 0.5444, 0.2566,
         0.9154, 0.6701, 0.7668, 0.4609, 0.5546, 0.1930, 0.0968, 0.3307, 0.7974,
         0.5901, 0.7050, 0.1784, 0.6846, 0.0941, 0.7879, 0.1097, 0.0439, 0.7162],
        [0.4384, 0.6009, 0.4982, 0.8722, 0.6400, 0.0966, 0.3675, 0.2985, 0.1202,
         0.5963, 0.0594, 0.8103, 0.3526, 0.4198, 0.6189, 0.5460, 0.6785, 0.5890,
         0.2087, 0.7020, 0.2193, 0.3720, 0.9307, 0.2834, 0.6664, 0.2207, 0.8420]])

In [14]:
# masking rate
mask_arr = rand<0.15 
mask_arr

tensor([[False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False,  True,  True, False,
          True, False, False, False, False, False, False],
        [False, False, False, False, False, False, False, False,  True, False,
         False,  True, False,  True, False,  True, False, False, False, False,
         False, False,  True, False, False,  True, False],
        [False, False, False,  True,  True, False, False, False, False, False,
         False, False, False, False, False,  True, False, False, False, False,
         False, False,  True, False,  True,  True, False],
        [False, False, False, False, False,  True, False, False,  True, False,
          True, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False]])

In [15]:
(inputs.input_ids != 101) * (inputs.input_ids != 102)

tensor([[False,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True, False,  True],
        [False,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True, False,  True,  True,  True,  True,  True],
        [False,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True, False],
        [False,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True, False,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True]])

In [16]:
# index position (select random index for masking)

selection = torch.flatten(mask_arr[0].nonzero()).tolist()
selection

[17, 18, 20]

In [17]:
# use the selection to select indeices (MASK token)

inputs.input_ids[0, selection] = 103
inputs.input_ids

# 103 = maksed token 

tensor([[  101,  1172, 33645, 12211, 13045, 27235, 34653, 51472, 18014, 46123,
         10609, 84416,   119, 21935,  1175, 32035, 12756,   103,   103,  1174,
           103, 40419,  1181, 93871, 24247,   102,     0],
        [  101, 12398, 19724, 21526, 10959, 11084, 26841, 11279, 12747, 15526,
         21841, 20773, 10422, 10946, 47468, 13045, 15245, 12727, 11724,   119,
         18287,   102,     0,     0,     0,     0,     0],
        [  101, 47529, 74831, 20687, 13413, 30462, 10609,  1175, 26179, 21596,
         43312, 35165, 82227,  1181, 36581, 21596, 43312, 12362,   119, 21526,
         10959, 11084,  1170, 50823, 21596, 35913,   102],
        [  101,  1170, 70724, 21596, 11112,  1174, 61701, 17861, 16584, 74190,
           119,   102,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0]])

In [18]:
# masked [2,14]
inputs.input_ids[0,2], inputs.input_ids[0,14]

(tensor(33645), tensor(1175))

In [19]:
outputs = model(**inputs)
outputs.keys()

odict_keys(['loss', 'logits'])

In [20]:
outputs.loss

tensor(6.9131, grad_fn=<NllLossBackward0>)

### Train with MLM

In [ ]:
# get wiki data 

!curl -c ./cookie -s -L "https://drive.google.com/uc?export=download&id=1zib1GI8Q5wV08TgYBa2GagqNh4jyfXZz" > /dev/null

!curl -Lb ./cookie "https://drive.google.com/uc?export=download&confirm=`awk '/download/ {print $NF}' ./cookie`&id=1zib1GI8Q5wV08TgYBa2GagqNh4jyfXZz" /wiki_20190620_small.txt

In [6]:
# load txt 

with open('/content/drive/MyDrive/STUDY/wiki_20190620_small (1).txt', 'r') as fp:
  text = fp.read().split('\n')

In [7]:
text[:5]

['제임스 얼 "지미" 카터 주니어는 민주당 출신 미국 39번째 대통령 이다.',
 '지미 카터는 조지아주 섬터 카운티 플레인스 마을에서 태어났다.',
 '조지아 공과대학교를 졸업하였다.',
 '그 후 해군에 들어가 전함·원자력·잠수함의 승무원으로 일하였다.',
 '1953년 미국 해군 대위로 예편하였고 이후 땅콩·면화 등을 가꿔 많은 돈을 벌었다.']

In [8]:
inputs = tokenizer(text, return_tensors='pt', max_length=128,
                   truncation=True,padding='max_length')

In [9]:
inputs

{'input_ids': tensor([[  101, 59354, 41625,  ...,     0,     0,     0],
        [  101,  1175, 32035,  ...,     0,     0,     0],
        [  101,  1175, 77072,  ...,     0,     0,     0],
        ...,
        [  101,  1169, 40389,  ...,     0,     0,     0],
        [  101,  1169, 40389,  ...,     0,     0,     0],
        [  101,   102,     0,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 0,  ..., 0, 0, 0]])}

In [10]:
# by cloning create labels
inputs['labels'] = inputs.input_ids.detach().clone()
inputs

{'input_ids': tensor([[  101, 59354, 41625,  ...,     0,     0,     0],
        [  101,  1175, 32035,  ...,     0,     0,     0],
        [  101,  1175, 77072,  ...,     0,     0,     0],
        ...,
        [  101,  1169, 40389,  ...,     0,     0,     0],
        [  101,  1169, 40389,  ...,     0,     0,     0],
        [  101,   102,     0,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 0,  ..., 0, 0, 0]]), 'labels': tensor([[  101, 59354, 41625,  ...,     0,     0,     0],
        [  101,  1175, 32035,  ...,     0,     0,     0],
        [  101,  1175, 

In [11]:
# create eaqual dim to inputs
rand = torch.rand(inputs.input_ids.shape)
rand.shape, inputs.input_ids.shape

(torch.Size([10001, 128]), torch.Size([10001, 128]))

In [12]:
rand

tensor([[0.1208, 0.8563, 0.9505,  ..., 0.2471, 0.1586, 0.6544],
        [0.6617, 0.3622, 0.7123,  ..., 0.4072, 0.6031, 0.0912],
        [0.4759, 0.6756, 0.4187,  ..., 0.7760, 0.8087, 0.9811],
        ...,
        [0.7781, 0.3052, 0.9047,  ..., 0.9661, 0.8660, 0.0066],
        [0.3209, 0.4924, 0.6045,  ..., 0.7260, 0.7963, 0.0031],
        [0.3399, 0.3704, 0.3566,  ..., 0.9922, 0.2191, 0.9294]])

In [13]:
# set masking rate 


# avoid 101,102,0 to be Masked
# (inputs.input_ids != 101) & (inputs.input_ids !=0) & (inputs.input_ids != 102)

mask_arr = ( rand < 0.15 ) * (inputs.input_ids != 101) * (inputs.input_ids !=0) * (inputs.input_ids != 102)
mask_arr
# True = Masked Tokens

tensor([[False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        ...,
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False]])

In [14]:
print("number of rows:",mask_arr.shape[0])

selection = []

for i in range(mask_arr.shape[0]):
  # flatten and get true values
  selection.append(torch.flatten(mask_arr[i].nonzero()).tolist())


# flatten and get true values


number of rows: 10001


In [15]:
# for each row what token will be masked 

selection[:10]

[[5, 17],
 [3, 7, 20, 23, 26],
 [],
 [3, 9, 12, 19],
 [7, 8, 11, 24, 26, 31],
 [11, 13],
 [1, 4, 24, 27, 29, 53, 57, 58, 63],
 [9, 10, 14, 28, 34],
 [3, 13, 14, 26],
 [1, 8, 11, 13, 14, 25, 41]]

In [16]:
for i in range(mask_arr.shape[0]):
  inputs.input_ids[i, selection[i]] = 103 # set mask token to 103

inputs.input_ids

tensor([[  101, 59354, 41625,  ...,     0,     0,     0],
        [  101,  1175, 32035,  ...,     0,     0,     0],
        [  101,  1175, 77072,  ...,     0,     0,     0],
        ...,
        [  101,  1169, 40389,  ...,     0,     0,     0],
        [  101,  1169, 40389,  ...,     0,     0,     0],
        [  101,   102,     0,  ...,     0,     0,     0]])

In [17]:
# convert to dataset object 
# define dataset class 

class CustomDataset(torch.utils.data.Dataset):
  def __init__(self,encodings):
    self.encodings = encodings
  
  def __getitem__(self,idx):
    return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
  
  def __len__(self):
    return len(self.encodings.input_ids)


In [18]:
# create object 
dataset = CustomDataset(inputs)

In [19]:
# create dataloader
dataloader = torch.utils.data.DataLoader(dataset, batch_size=16,shuffle=True)

In [20]:
# connect cuda 
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(105879, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=T

In [34]:
# activate train 
model.train().to(device)

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(105879, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=T

In [22]:
# optimizer 

from transformers import AdamW

optim = AdamW(model.parameters(), lr = 3e-5)

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [23]:
epochs =2 

for epoch in range(epochs):
  loop = tqdm(dataloader, leave=True)
  
  for batch in loop:
    
    # calculate grad 
    optim.zero_grad()
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['input_ids'].to(device)
    labels = batch['input_ids'].to(device)

    outputs = model(input_ids, attention_mask = attention_mask,
                    labels =labels)
    loss = outputs.loss
    loss.backward()
    optim.step()

    loop.set_description(f'Epoch {epoch}')
    loop.set_postfix(loss=loss.item())



  0%|          | 0/626 [00:00<?, ?it/s]

<ipython-input-17-589823d368b1>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


  0%|          | 0/626 [00:00<?, ?it/s]

In [38]:
from transformers import pipeline
fill_mask = pipeline('fill-mask', top_k=5, model=model, tokenizer=tokenizer)
fill_mask("이순신은[MASK]중기의 무신이다.")

RuntimeError: ignored

In [24]:
! nvidia-smi

Fri Jan  6 10:55:28 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   71C    P0    31W /  70W |   9754MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [46]:
import gc
gc.collect()
torch.cuda.empty_cache()